# Crawling PTA dan Berita

### Library

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

## 1. Crawling PTA

In [17]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

### Fungsi

In [8]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # Pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1

In [10]:
# Contoh pemakaian
print(get_max_page(10))

172


In [11]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

### Fungsi Crawling Semua Data PTA

In [23]:
def pta_all():
    start_time = time.time()

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak_id": [],
        "abstrak_en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": []
    }

    total_prodi = 2
    total_pages = 0
    max_pages_dict = {}

    # hitung total halaman (untuk tiap prodi)
    for i in range(1, total_prodi + 1):
        max_page = get_max_page(i)
        max_pages_dict[i] = max_page
        total_pages += max_page

    for i in range(1, total_prodi + 1):
        max_page = max_pages_dict[i]
        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA (angka terakhir di URL)
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip()
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi)

            # update progress bar per prodi
            print_progress(i, prodi, j, max_page)

        sys.stdout.write("\n")  # pindah baris setelah 1 prodi selesai

    # simpan ke CSV
    df = pd.DataFrame(data)
    df.to_csv("pta_all.csv", index=False, encoding="utf-8-sig")

    # hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [24]:
pta_all()

[1] Ilmu Hukum - Page 284/284 [████████████████████] 100.00%


[2] Teknologi Industri Pertanian - Page 114/114 [████████████████████] 100.00%



✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 1985
⏱️ Waktu eksekusi: 0 jam 52 menit 19 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
1980,160331100056,MUHAMMAD RIZAL,PENGENDALIAN KUALITAS TEMBAKAU KERING DENGAN ...,Pengendalian kualitas merupakan sebuah upaya u...,Quality control is an effort to maintain and i...,"Dr. M. Fuad Fauzul M, STP., M.Si","Ir. Moh. Fakhry., MP",Teknologi Industri Pertanian
1981,160331100079,ACHMAD ARDI WIJAYA,PENGEMBANGAN PRODUK KERUPUK AMPLANG DENGAN PEN...,Produk ramah lingkungan dapat mengurangi terja...,Environmentally friendly products can reduce t...,"Dr. Moh. Fuad Fauzul M, S.TP., M.Si","Khoirul Hidayat, ST., MT",Teknologi Industri Pertanian
1982,130331100082,Hendriyadi,Analisis Faktor Yang Mempengaruhi Pembelian Te...,UD. Dianis merupakan industri rumah tangga yan...,UD. Dianis is a home industry that produces tr...,"Dr. Moh. Fuad Fauzul M, S.TP., M.Si","Khoirul Hidayat, ST.,MT",Teknologi Industri Pertanian
1983,160331100032,Peni Asrurin,Karakteristik Fisik dan Sensoris Mi Kering Tep...,ABSTRAK\n\n\n Mi merupakan makanan yang sudah ...,ABSTRACT\n\n\nNoodles are foods that commonly ...,"Ir. Umi Purwandari, M.App.Sc., Ph.D.","Cahyo Indarto, S.TP., M.P",Teknologi Industri Pertanian


### Fungsi Crawling Data PTA (Dengan Batas Page)

In [19]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak id": [],
        "abstrak en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": [],
    }

    for i in range(1, 42):  # jumlah prodi
        total_pages = 3  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA
                id_match = re.search(r"/detail/(\d+)", link)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                # Judul
                judul = isi.select_one('a.title').text

                # Penulis
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]

                # Pembimbing Pertama
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]

                # Pembimbing Kedua
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                # Abstrak
                paragraf = isi.select('p[align="justify"]')
                abstrak = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstract = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                # simpan data
                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["abstrak id"].append(abstrak)
                data["abstrak en"].append(abstract)
                data["prodi"].append(prodi)

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [20]:
pta()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,id,penulis,judul,abstrak id,abstrak en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
476,160281100013,"Lisa Sri rahmatullah, S. Sos. I",Dampak Sosial Ekonomi Pariwisata Religi Makam ...,Penelitian ini bertujuan untuk mengetahui baga...,The purpose of this study is to analyze the so...,"Dr. Diah Wahyuningsih, S.E., M.Si.","Dr. Eni Sri Rahayuningsih, S.E., M.E.",Magister Ilmu Ekonomi
477,160281100002,Indah Ainun Nikmah,Peranan Zakat Produktif Dalam Meningkatkan Eko...,Peranan Zakat Produktif dalam Meningkatkan Eko...,The Role of Productive Zakat in Improving Must...,"Dr. Kurniyati Indahsari, M.Si","Dr. Abdur Rahman, S.Ag. MEI",Magister Ilmu Ekonomi
478,170361100010,ahmad syaiful umam,KARAKTERISASI DAN KOLEKSI PLASMA NUTFAH UNTUK ...,Madura merupakan salah satu wilayah pemasok ko...,Madura is one of the regions supplying horticu...,"Dr. Ir. Gita Pawana, M.Si","Dr. Ir. Hj. SIti Fatimah, M.Si",Magister Pengelolaan Sumber Daya Alam
479,170361100001,Siti Holifah,PENGOLAHAN LIMBAH AIR REBUSAN IKAN TERI MENJAD...,Ikan Teri perlu penanganan serius pasca panen ...,Anchovy needs serious handling after harvest b...,"Dr.Apri Arisandi,S.Pi.,M.Si.","Dr.Ir.H.Asfan,MP.",Magister Pengelolaan Sumber Daya Alam


## Page & Link Keluar PTA

In [21]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta_links():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "no": [],
        "page": [],
        "link_keluar": []
    }

    no = 1  # nomor urut

    for i in range(1, 42):  # jumlah prodi
        total_pages = 3  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                data["no"].append(no)
                data["page"].append(url)          # link page
                data["link_keluar"].append(link)  # link detail
                no += 1

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [22]:
pta_links()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,no,page,link_keluar
0,1,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
1,2,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
2,3,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
3,4,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/090...
4,5,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
...,...,...,...
476,477,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
477,478,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
478,479,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...
479,480,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...


## 2. Crawling Berita

### Fungsi

In [2]:
# Fungsi progress bar manual
def print_progress(kategori, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r{kategori} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

In [3]:
# Fungsi untuk ambil konten artikel
def get_article_content(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    paragraphs = []
    content_divs = soup.find_all("div", id="content-wrapper")
    for div in content_divs:
        for p in div.find_all("p"):
            text = p.get_text(strip=True)
            if text and not text.lower().startswith("baca juga"):
                paragraphs.append(text)
    return " ".join(paragraphs)

### Fungsi Crawling Berita

In [4]:
def berita(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "judul_berita": [],
        "isi_berita": [],
        "kategori_berita": []
    }

    for cat_id, cat in enumerate(categories, start=1):
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                title = a.get_text(strip=True)

                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                try:
                    content = get_article_content(link)
                except:
                    content = ""

                data["id_berita"].append(berita_id)
                data["judul_berita"].append(title)
                data["isi_berita"].append(content)
                data["kategori_berita"].append(cat)

            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_berita.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "lingkungan", "hiburan", "internasional", "otomotif", "olahraga", "sepakbola"]

In [5]:
berita(categories, pages_per_category=5)

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

lingkungan - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%

sepakbola - Page 5/5 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 900
⏱️ Waktu eksekusi: 0 jam 17 menit 34 detik


,id_berita,judul_berita,isi_berita,kategori_berita
0,2068508,"Setelah Rapat dengan DPR, Menteri Keuangan Tem...",MENTERI Keuangan Purbaya Yudhi Sadewa menemui ...,politik
1,2068499,Mahasiswa Tasikmalaya Desak Penghapusan Dana P...,"MAHASISWA Kota Tasikmalaya, Jawa Barat, mengge...",politik
2,2068489,DPR Harap Menpora Baru Tinjau Ulang Permenpora...,KOMISI X Dewan Perwakilan Rakyat (DPR) berhara...,politik
3,2068485,"Dasco Temui Prabowo di Istana, Laporkan Keputu...",PRESIDENPrabowoSubianto bertemu dengan Wakil K...,politik
4,2068478,Sengketa Pilkada Kabupaten Boven Digoel Kandas...,DUA permohonan sengketa pemilihan kepala daera...,politik
...,...,...,...,...
895,2066144,Pesan Alexander Isak kepada Newcastle Usai Sag...,PEMAIN sepak bolaAlexander Isakberterima kasih...,sepakbola
896,2066121,Prediksi Indonesia vs Laos di Kualifikasi Pial...,DUEL Timnas U-23Indonesia vs Laosakan tersaji ...,sepakbola
897,2066036,Alasan Patrick Kluivert Puji Thom Haye dan Eli...,PELATIHTimnas IndonesiaPatrick Kluivert menila...,sepakbola
898,2066051,Daftar 23 Pemain Manchester City untuk Liga Ch...,BURSA transfer musim panas sudah ditutup.Manch...,sepakbola


## Page & Link Keluar Berita

In [ ]:
def berita_links(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "page": [],
        "link_keluar": []
    }

    for cat in categories:
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                data["id_berita"].append(berita_id)
                data["page"].append(url)         # link page
                data["link_keluar"].append(link) # link detail

            # update progress bar
            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_links.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "lingkungan", "hiburan", "internasional", "otomotif", "olahraga", "sepakbola"]

In [ ]:
berita_links(categories, pages_per_category=5)

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

lingkungan - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%

sepakbola - Page 5/5 [████████████████████] 100.00%


✅ Seluruh link berhasil dikumpulkan!
📊 Total entri: 900
⏱️ Waktu eksekusi: 0 jam 1 menit 4 detik


,id_berita,page,link_keluar
0,2068198,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/respons-abdul-kad...
1,2068188,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/menkes-budi-gunad...
2,2068167,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/penyusunan-ruu-pe...
3,2068149,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/bahlil-kirim-tim-...
4,2068145,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/koalisi-sipil-sor...
...,...,...,...
895,2065734,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/calvin-verdonk-...
896,2065727,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/penyebab-bayer-...
897,2065643,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/calvin-verdonk-...
898,2065621,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/kualifikasi-pia...
